# IMPORTANDO BIBLIOTECAS

In [ ]:
# Importando bibliotecas importantes
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score
)
from sklearn.model_selection import (
    StratifiedKFold,
    cross_validate,
    train_test_split
)
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

from joblib import Parallel, delayed


# EXIBIÇÃO DOS DATAFRAMES

In [ ]:
# Configurando a exibição do dataframe
# A seguir estão configurações relacionadas à exibição de DataFrames no Pandas

# Define o número máximo de colunas a serem exibidas sem truncamento
pd.set_option('display.max_columns', None)

# Define se as colunas do DataFrame podem ser expandidas para ajustar a largura da tela
pd.set_option('display.expand_frame_repr', False)

# Define a largura máxima de coluna para exibição sem truncamento
pd.set_option('display.max_colwidth', None)

# Define o número máximo de linhas a serem exibidas ao imprimir o DataFrame
pd.set_option('display.max_rows', 30)


# LEITURA DOS ARQUIVOS

In [ ]:
# Lista com caminhos locais distintos
diretorios_locais = [
    r'C:\Users\brand\OneDrive\Área de Trabalho\Fraud_Detection_ML\Data\\',
    r'C:\Users\Giovanni Maia\Desktop\Dados\\'
]

diretorio_github= 'https://media.githubusercontent.com/media/jeanbrandao01/Fraud_Detection_ML/dev/Data/'

def carregar_dataframes(diretorios_locais, diretorio_github):
    for diretorio_local in diretorios_locais:
        try:
            # Tente carregar o DataFrame de fraudTest.csv a partir do diretório local
            df_test = pd.read_csv(diretorio_local + 'fraudTest.csv')

            # Tente carregar o DataFrame de fraudTrain.csv a partir do diretório local
            df_train = pd.read_csv(diretorio_local + 'fraudTrain.csv')

            # Se o carregamento for bem-sucedido, retorne os DataFrames
            return df_test, df_train
        except FileNotFoundError:
            continue  # Continue para o próximo diretório local em caso de erro

    # Se nenhum diretório local contiver os arquivos, carregue do GitHub
    url_test = diretorio_github + 'fraudTest.csv'
    url_train = diretorio_github + 'fraudTrain.csv'

    df_test = pd.read_csv(url_test)
    df_train = pd.read_csv(url_train)

    return df_test, df_train

df_test, df_train = carregar_dataframes(diretorios_locais,diretorio_github)


# EXIBIÇÃO DOS DATAFRAMES ORIGINAIS

In [ ]:
# Dataframe "fraudTest.csv"
df_test.head()

In [ ]:
# Dataframe "fraudTrain.csv"
df_train.head()

# ANÁLISE DESCRITIVA: DATASET DE TREINO - FRAUDTRAIN.CSV

In [ ]:
# Algumas estatisticas do dataset
df_train.describe()

In [ ]:
# Verificando as colunas existentes
df_train.info()

In [ ]:
# Realizando a contagem de valores da coluna "merchant"
df_train.merchant.value_counts()

In [ ]:
# Realizando a contagem de valores da coluna "is_fraud"
print("Contagem de valores da coluna is_fraud:")
print("-"*39)
print(df_train.is_fraud.value_counts())


In [ ]:
# Verificando a extensão do dataset
print("Extensão do dataset:")
print("-"*23)
print(df_train.shape)


In [ ]:
# Verificando os nomes de cada coluna 
print("Colunas do dataset")
print(df_train.columns)

# ANÁLISE EXPLORATÓRIA: DATASET DE TREINO - FRAUDTRAIN.CSV

In [ ]:
# Calcular a proporção de fraudes
fraud_proportion = df_train['is_fraud'].value_counts(normalize=True)

# Criar um DataFrame para facilitar a criação do gráfico
fraud_data = pd.DataFrame({'Classe de Fraude': fraud_proportion.index, 'Proporção': fraud_proportion.values})

# Criar o gráfico de barras com o Plotly
fig = px.bar(fraud_data, x='Classe de Fraude', y='Proporção', text='Proporção', title='Distribuição de Fraudes', labels={'Classe de Fraude': 'Classe de Fraude'})
fig.update_traces(texttemplate='%{text:.2%}', textposition='outside')
fig.show()



In [ ]:
# mostrando a porcentagem de transações fraudulentas e não fraudulentas para cada categoria presente na coluna category 
round(pd.crosstab(index=df_train.category, columns=df_train.is_fraud, normalize='index')*100,2)

In [ ]:
# Cria uma tabela de contingência
crosstab = pd.crosstab(index=df_train.category, columns=df_train.is_fraud, normalize='index') * 100

# Reinicia o índice para facilitar o manuseio
crosstab = crosstab.reset_index()

# Cria um gráfico de barras empilhadas usando o Plotly Express
fig = px.bar(crosstab, x='category', y=[0, 1], title="Gráfico de Barras Empilhadas da Tabela de Contingência (Percentual)",
             labels={0: "Não Fraude", 1: "Fraude"},
             height=400)

# Atualiza o layout para um gráfico de barras empilhadas
fig.update_layout(barmode='stack')

# Mostra o gráfico
fig.show()


In [ ]:
# Agrupa os dados por estado e calcula o número total de transações em cada estado
total_transacoes_por_estado = df_train["state"].value_counts()

# Filtra os dados para transações fraudulentas
dados_fraudulentos = df_train[df_train["is_fraud"] == 1]

# Agrupa os dados fraudulentos por estado e calcula o número de transações fraudulentas em cada estado
transacoes_fraudulentas_por_estado = dados_fraudulentos["state"].value_counts()

# Calcula a taxa de fraude para cada estado (transações fraudulentas / total de transações)
taxa_de_fraude = (transacoes_fraudulentas_por_estado / total_transacoes_por_estado).fillna(0)

# Reinicia o índice e ordena os resultados por 'count' de forma descendente
taxa_de_fraude.reset_index().sort_values(by='count', ascending=False)


In [ ]:
# Retorna as últimas 20 linhas (estados) dessa Series, representando os estados com o menor número de transações. 
total_transacoes_por_estado.tail(20)

In [ ]:
# Retorna o número de cidades únicas presentes na coluna cidade
df_train.city.nunique()

In [ ]:
# Agrupa os dados por cidade e calcula o número total de transações em cada cidade
total_transacoes_por_cidade = df_train["city"].value_counts()

# Filtra os dados para transações fraudulentas
dados_fraudulentos = df_train[df_train["is_fraud"] == 1]

# Agrupa os dados fraudulentos por cidade e calcula o número de transações fraudulentas em cada cidade
transacoes_fraudulentas_por_cidade = dados_fraudulentos["city"].value_counts()

# Calcula a taxa de fraude para cada cidade (transações fraudulentas / total de transações)
taxa_de_fraude = (transacoes_fraudulentas_por_cidade / total_transacoes_por_cidade).fillna(0)

# Seleciona as cidades com taxa de fraude igual a 1, reinicia o índice e ordena os resultados por 'count' de forma descendente
taxa_de_fraude[taxa_de_fraude == 1].reset_index().sort_values(by='count', ascending=False)


In [ ]:
# Define o estilo (opcional)
sns.set_style("whitegrid")

# Cria um violin plot usando o Seaborn para valor da compra por classe
plt.figure(figsize=(10, 6))  # Define o tamanho da figura (opcional)
sns.violinplot(data=df_train.query('amt <= 200'), x='is_fraud', y='amt')
# Define o título e rótulos
plt.title('Violin Plot - Distribuição do Valor da Compra por Classe de Fraude')
plt.xlabel('Classe de Fraude')
plt.ylabel('Valor da Compra')

# Mostra o gráfico
plt.show()


In [ ]:
# Define o estilo (opcional)
sns.set_style("whitegrid")

# Cria um boxplot usando o Seaborn para distruibuição do valor de compra
plt.figure(figsize=(10, 6))  # Define o tamanho da figura (opcional)
sns.boxplot(data=df_train.query('amt <= 200'), y='amt', color='skyblue')  # Adicionado cor para melhor visualização

# Define o título e rótulos
plt.title('Boxplot - Distribuição do Valor da Compra')
plt.ylabel('Valor da Compra')

# Mostra o gráfico
plt.show()


In [ ]:
# Define o estilo (opcional)
sns.set_style("whitegrid")

# Escolhe uma paleta de cores para as barras (cores mais claras)
bar_palette = sns.color_palette("Blues", as_cmap=True)

# Escolhe uma cor mais escura para a linha do KDE
line_color = 'navy'

# Cria um histograma com KDE usando o Seaborn para valor de compra
plt.figure(figsize=(10, 6))  # Define o tamanho da figura (opcional)
sns.histplot(data=df_train.query('amt <= 200'), x='amt', kde=True)
sns.kdeplot(data=df_train.query('amt <= 200')['amt'], color=line_color, linewidth=2)

# Define o título e rótulos
plt.title('Histograma com KDE - Valor da Compra')
plt.xlabel('Valor da Compra')
plt.ylabel('Contagem')

# Mostra o gráfico
plt.show()


In [ ]:
# Define o estilo (opcional)
sns.set_style("whitegrid")

# Cria um histograma com KDE usando o Seaborn para valores de compra acima de 2000
plt.figure(figsize=(10, 6))  # Define o tamanho da figura (opcional)
sns.histplot(data=df_train.query('amt > 2000'), x='amt', kde=True, color='red')  # Adiciona cor para melhor visualização

# Define o título e rótulos
plt.title('Histograma com KDE - Valor de Compra Acima de 2000')
plt.xlabel('Valor de Compra')
plt.ylabel('Contagem')

# Mostra o gráfico
plt.show()


In [ ]:
# Selecionar apenas as colunas numéricas
numeric_columns = df_train.select_dtypes(include=['int64', 'float64'])

# Calculando a matriz de correlação para colunas numéricas
correlation_matrix = numeric_columns.corr()

# Criando uma figura maior
plt.figure(figsize=(20, 10))

# Criando um mapa de calor de correlação com paleta de cores personalizada
sns.heatmap(correlation_matrix, annot=True, cmap='viridis', center=0, cbar_kws={'label': 'Correlação'})
plt.title('Mapa de Calor de Correlação')

plt.show()


In [ ]:
# Análise temporal

# Convertendo a coluna 'trans_date_trans_time' para um objeto de data e hora
df_train['trans_date_trans_time'] = pd.to_datetime(df_train['trans_date_trans_time'])

# Extraindo características de data e hora
df_train['ano'] = df_train['trans_date_trans_time'].dt.year
df_train['mes'] = df_train['trans_date_trans_time'].dt.month
df_train['dia_da_semana'] = df_train['trans_date_trans_time'].dt.day_name()
df_train['hora'] = df_train['trans_date_trans_time'].dt.hour
df_train['minuto'] = df_train['trans_date_trans_time'].dt.minute
df_train['segundo'] = df_train['trans_date_trans_time'].dt.second

# Filtrando fraudes
frauds = df_train[df_train['is_fraud'] == 1]

# Contando fraudes por mês
frauds_by_month = frauds['mes'].value_counts().sort_index().reset_index()
frauds_by_month.columns = ['Mês', 'Número de Fraudes']

# Criando um gráfico de barras interativo com Plotly
fig = px.bar(frauds_by_month, x='Mês', y='Número de Fraudes', title='Fraudes por Mês',
             labels={'Mês': 'Mês', 'Número de Fraudes': 'Número de Fraudes'})

# Adicionando números em cima das barras
fig.update_traces(text=frauds_by_month['Número de Fraudes'], textposition='outside')

fig.show()


In [ ]:
# Contagem de ocorrências por categoria
category_counts = df_train['category'].value_counts().reset_index()
category_counts.columns = ['Categoria', 'Contagem']

# Agrupando por categoria e calculando a proporção de fraudes
fraud_proportion_by_category = df_train.groupby('category')['is_fraud'].mean().reset_index()
fraud_proportion_by_category.columns = ['Categoria', 'Proporção de Fraudes']

# Criando um gráfico de barras interativo com Plotly
fig = px.bar(fraud_proportion_by_category, x='Categoria', y='Proporção de Fraudes', title='Proporção de Fraudes por Categoria',
             labels={'Categoria': 'Categoria', 'Proporção de Fraudes': 'Proporção de Fraudes'})

# Formatando os números com cinco casas decimais
fraud_proportion_by_category['Proporção de Fraudes'] = fraud_proportion_by_category['Proporção de Fraudes'].round(5)

# Adicionando os números formatados (com cinco casas decimais) em cima das barras
fig.update_traces(text=fraud_proportion_by_category['Proporção de Fraudes'], textposition='outside')

fig.show()


# FEATURE ENGINEERING

In [ ]:
df_train.head()

In [ ]:
df_train['trans_date_trans_time'] = pd.to_datetime(df_train['trans_date_trans_time'])


df_train.sort_values(by=['cc_num' ,'trans_date_trans_time'], ascending=True, inplace=True)


agrupado_cc_valor = df_train.groupby('cc_num')['amt']

media_cartao = agrupado_cc_valor.transform('mean')
desv_pad_cartao = agrupado_cc_valor.transform('std')

df_train['media'] = media_cartao

df_train['amt_score_z'] = (df_train['amt'] - media_cartao) / desv_pad_cartao

df_train.head()

In [ ]:
df_train.loc[df_train.is_fraud == 0, 'amt_score_z'].mean()

In [ ]:
df_train.select_dtypes(exclude='object').corr(method='pearson')

# FUNÇÕES E CLASSES

In [ ]:
# Adicione uma constante pequena antes de aplicar o log
def calculate_woe(df, feature, target):
    df = df[[feature, target]].copy()
    df['n_total'] = 1
    df = df.groupby([feature, target]).count().unstack().fillna(0)
    df.columns = df.columns.droplevel()
    df['n_event'] = df[1]
    df['n_non_event'] = df[0]
    df['n_total'] = df['n_event'] + df['n_non_event']
    df['event_rate'] = df['n_event'] / df['n_event'].sum()
    df['non_event_rate'] = df['n_non_event'] / df['n_non_event'].sum()
    # Adicione uma constante pequena para evitar log(0)
    df['woe'] = np.log((df['event_rate'] + 1e-9) / (df['non_event_rate'] + 1e-9))
    woe_dict = df['woe'].to_dict()
    return woe_dict


In [ ]:
# Classe do transformador WoE (Weight of Evidence)
class WoETransformer(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_features):
        self.categorical_features = categorical_features

    def fit(self, X, y=None):
        self.woe_dict_ = {}
        for column in self.categorical_features:
            woe_values = calculate_woe(pd.DataFrame({column: X[column], 'target': y}), column, 'target')
            self.woe_dict_[column] = woe_values
        return self

    def transform(self, X):
        X_transformed = X.copy()
        for column, woe_values in self.woe_dict_.items():
            X_transformed[column] = X_transformed[column].map(woe_values)
        return X_transformed


# TREINAMENTO E AVALIAÇÃO DO MODELO

In [ ]:
# Bloco 2: Função para treinar e prever para cada classificador
def train_and_predict(classifier, X_train_transformed, y_train, X_test_transformed):
    pipe = Pipeline(steps=[('classifier', classifier)])
    pipe.fit(X_train_transformed, y_train)
    y_pred = pipe.predict(X_test_transformed)
    return classifier.__class__.__name__, y_pred


In [ ]:
# Bloco 3: Função para avaliar métricas de desempenho
def evaluate_metrics(y_true, y_pred, classifier_name):
    accuracy = accuracy_score(y_true, y_pred)   
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    result = {
        "Classifier": classifier_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1,
    }
    return result


In [ ]:
# Bloco 4: Função principal que treina, avalia e prevê para cada classificador
def train_evaluate_and_predict(classifier, X_train_transformed, y_train, X_test_transformed, y_test):
    classifier_name, y_pred = train_and_predict(classifier, X_train_transformed, y_train, X_test_transformed)
    metrics_result = evaluate_metrics(y_test, y_pred, classifier_name)
    return classifier_name, metrics_result


# PRÉ PROCESSAMENTO E CRIAÇÃO DO PIPELINE

In [ ]:
# Bloco 5: Pré-processamento e Treinamento do Modelo
# Substitua 'df_train' pelos seus dados reais
features = df_train[['amt', 'category', 'gender', 'city', 'state', 'zip', 'lat', 'long', 'unix_time', 'merch_lat', 'merch_long', 'ano', 'mes', 'dia_da_semana', 'hora', 'minuto', 'segundo']]
labels = df_train['is_fraud']



In [ ]:

# Divisão dos dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

In [ ]:
# Lista de features numéricas contínuas e categóricas
numericas_continuas = ['amt', 'zip', 'lat', 'long', 'unix_time', 'merch_lat', 'merch_long', 'ano', 'mes', 'hora', 'minuto', 'segundo']
string_categoricas = ['category', 'gender', 'city', 'state', 'dia_da_semana']

In [ ]:
# Transformador para features numéricas contínuas
numeric_transformer = StandardScaler()

# Transformador para features categóricas usando a classe WoETransformer
categorical_transformer = WoETransformer(categorical_features=string_categoricas)

In [ ]:
# Lista de transformadores a serem aplicados a cada conjunto de features
transformers = [
    ('num_continuas', numeric_transformer, numericas_continuas),
    ('str_categoricas', categorical_transformer, string_categoricas)
]

In [ ]:
# Criação do ColumnTransformer integrado ao Pipeline
preprocessor = ColumnTransformer(
    transformers=transformers,
    remainder='passthrough'
)

In [ ]:
# Integrando o ColumnTransformer ao Pipeline com Regressão Logística (substitua conforme necessário)
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', RandomForestClassifier())])

In [ ]:

# Treinamento do pré-processador nos dados de treino
X_train_transformed = preprocessor.fit_transform(X_train, y_train)

In [ ]:

# Treinamento do modelo completo
pipeline.fit(X_train, y_train)

In [ ]:
# Aplicação do pré-processador nos dados de teste
X_test_transformed = preprocessor.transform(X_test)

In [ ]:
# Previsão nos dados de teste
y_pred = pipeline.predict(X_test)

# AVALIAÇÃO EM PARALELO PARA TODOS OS CLASSIFICADORES

In [ ]:
# Bloco 6: Avaliação em Paralelo para Todos os Classificadores
# Lista de classificadores
classifiers_list = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
]


In [ ]:
# Execução em paralelo para todos os classificadores - Métricas
metrics_results = Parallel(n_jobs=-1)(
    delayed(train_evaluate_and_predict)(
        classifier, X_train_transformed, y_train, X_test_transformed, y_test
    ) for classifier in classifiers_list
)

In [ ]:
# Definindo classifier_names
classifier_names = [classifier.__class__.__name__ for classifier in classifiers_list]

# Organizando os resultados
metrics_dict = {'Classifier': [], 'Accuracy': [], 'Precision': [], 'Recall': [], 'F1-score': []}
for result in metrics_results:
    classifier_name, metrics_result = result
    metrics_dict['Classifier'].append(classifier_name)
    metrics_dict['Accuracy'].append(metrics_result['Accuracy'])
    metrics_dict['Precision'].append(metrics_result['Precision'])
    metrics_dict['Recall'].append(metrics_result['Recall'])
    metrics_dict['F1-score'].append(metrics_result['F1-score'])


In [ ]:
# Criando DataFrame com os resultados
metrics_df = pd.DataFrame(metrics_dict)
# Exibindo o DataFrame
print(metrics_df)


In [ ]:
# Melhor classificador

# Especificar a métrica pela qual queremos classificar os modelos (por exemplo, F1-score)
metric_to_maximize = 'F1-score'

# Classificar o DataFrame pelo valor máximo da métrica
sorted_df = metrics_df.sort_values(by=metric_to_maximize, ascending=False)

# Exibir o classificador que obteve o melhor desempenho
best_classifier = sorted_df.iloc[0]['Classifier']
print(f"O melhor classificador com base na métrica {metric_to_maximize} é: {best_classifier}")


In [ ]:
# Execução em paralelo para todos os classificadores - Previsões
results = Parallel(n_jobs=-1)(
    delayed(train_and_predict)(
        classifier, X_train_transformed, y_train, X_test_transformed
    ) for classifier in classifiers_list
)

In [ ]:
# Descompactando a lista de resultados
classifier_names, predictions = zip(*results)

# Criando um dicionário com os resultados
data_dict = {"is_fraud": y_test}

# Adicionando as previsões de cada classificador ao dicionário
for classifier_name, prediction in zip(classifier_names, predictions):
    data_dict[classifier_name] = prediction

# Criando um DataFrame a partir do dicionário
results_df = pd.DataFrame(data_dict)

# Exibindo o DataFrame
print(results_df)


# CROSS VALIDATION

In [ ]:
# Criar o pipeline com o classificador
classifier_cv = RandomForestClassifier()

# Criar o pipeline com o classificador
pipe_cv = Pipeline(steps=[('classifier', classifier_cv)])

# Definir a estratégia de validação cruzada (Stratified K-Fold)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Definir as métricas desejadas
scoring_metrics = ['accuracy', 'precision', 'recall', 'f1']

# Executar a validação cruzada e calcular as métricas
cv_results = cross_validate(pipe_cv, X_train_transformed, y_train, cv=stratified_kfold, scoring=scoring_metrics)

# Exibir os resultados
for metric in scoring_metrics:
    print(f"{metric.capitalize()} médio na validação cruzada:", cv_results[f'test_{metric}'].mean())
    print(f"Desvio padrão dos {metric.capitalize()} na validação cruzada:", cv_results[f'test_{metric}'].std())
    print("---")

# MATRIZ DE CONFUSÃO PARA TODOS OS INDICADORES (Não usei a cv pra esse conjunto, apenas a validação normal)

In [ ]:
# Lista de todas as colunas de previsão
prediction_columns = [
    'LogisticRegression',
    'DecisionTreeClassifier',
    'RandomForestClassifier',
    'AdaBoostClassifier',
    'GradientBoostingClassifier'
]

# Loop para criar matrizes de confusão interativas para cada classificador
for classifier_column in prediction_columns:
    # Criar a matriz de confusão
    conf_matrix = confusion_matrix(results_df['is_fraud'], results_df[classifier_column])

    # Configurações do heatmap
    heatmap = ff.create_annotated_heatmap(
        z=conf_matrix,
        x=['Not Fraud', 'Fraud'],
        y=['Not Fraud', 'Fraud'],
        colorscale='Blues'
    )

    # Atualizar layout para adicionar rótulos
    heatmap.update_layout(
        xaxis=dict(title='Predicted'),
        yaxis=dict(title='True'),
        title=f'Confusion Matrix: is_fraud vs. {classifier_column}'
    )

    # Exibir o gráfico
    heatmap.show()


# APLICANDO A FUNÇÃO DE TREINO NO DE TESTE